In [1]:
# import pandas

import pandas as pd

In [2]:
# Read in the dataset
hotelBookings = pd.read_csv("hotel_bookings.csv")

In [ ]:
# First look at the dataset
hotelBookings

### List Columns


Using the `.columns` attribute to display an `Index` object containing the name of all the columns.<br>

*The `Index` object looks like a list but **IT IS NOT** a list* <br>

Pandas uses Index objects because they provide:<br>

- Immutability (you can’t accidentally change it)<br>

- Fast lookups<br>

- Alignment behavior when joining/merging<br>

- Set-like operations (intersection, difference, union)<br>

- Ability to be used as row labels and column labels<br>

A normal list can’t do this.

See __[here](https://pandas.pydata.org/docs/reference/api/pandas.Index.html)__




In [ ]:
hotelBookings.columns

# OPTIONAL: Use a for loop to display column names in a more organized manner:

for eachColumn in hotelBookings.columns:
    print(eachColumn)

### Drop/Rename Columns

To drop columns we will use the `.drop()` function that, for this exercise, will take two parameters:<br>
    - `columns`, that specify the columns to be dropped <br>
    - `inplace`, that, when set to `True`, applies the drop directly to the existing DataFrame, while `inplace=False` returns a new DataFrame and leaves the original untouched.<br>

To rename columns we will use the `.rename()` function that, for this exercise, will take the following parameters:<br>
    - a dictionary where each **key** is the original column name and each **value** is the new column name.<br>
    <br>
      `df.rename({'OldName': 'NewName'})`.<br>
    <br>
    - `axis=1` that specifies whether to drop labels from the index (0 or ‘index’) or columns (1 or ‘columns’).<br>
    - `inplace=True`... 

In [ ]:
hotelBookings.drop(columns=['required_car_parking_spaces'], inplace=True)
hotelBookings.rename({'adults': 'count_of_adults',
                      'children': 'count_of_children',
                      'babies': 'count_of_babies'}, axis=1, inplace=True)
hotelBookings.head(5)

### Handling missing values (NaNs)

- Check what columns have `NaN` or `null` values;
- Check percentage of `null` values in each column;
- Make a determination of:
    1. which columns should be dropped based on the amount of `null` values in them using `.dropna()`;
    2. which columns could be filled in based on values from other columns using the `.unique()` function;
    3. Single out one unique value using comparison and look for similarities in columns;
- If columns can be filled in: GREAT! Else, use the `fillna()` method to replace `NaN` with a placeholder: `None`, for string values, `pd.NA` for boolean values and `-1` for numerical values.
- For columns with a small amount of `NaN` values, like the `count_of_children` column, use `isna()` to isolate the rows containing `NaN` values.

In [ ]:
# OPTIONAL: Display as percentage => BE CAREFUL BECAUSE THIS WILL RETURN A STRING, THEREFORE UNABLE TO PERFORM CALCULATIONS
((hotelBookings.isnull().sum() * 100 / len(hotelBookings)) # AI Generated Code
    .round(2) # rounds two decimal points
    .astype(str) + '%') # converts it into string so it can display '%'

hotelBookings.isnull().sum() * 100 / len(hotelBookings)


In [ ]:
# Checking unique values in a certain column and using a placeholder to replace null values

hotelBookings['agent'].unique()

hotelBookings[hotelBookings['agent'] == 5]

# hotelBookings['agent'].fillna(-1, inplace=True) #inplace=True at the end of the chain does not modify the original dataframe.
# FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
# The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

# For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.

hotelBookings['agent'] = hotelBookings['agent'].fillna(-1) #AI Generated Code

display(hotelBookings)


In [ ]:
# Isolating columns with a few rows containing null values

hotelBookings[hotelBookings['count_of_children'].isna()] # There are four rows => these rows can be dropped

hotelBookings.dropna(subset=['count_of_children'], inplace=True) 

display(hotelBookings)


In [9]:
hotelBookings[hotelBookings['country'].isna()] # Same concept as above... In this case there are 488 rows. Too much to be excluded. 

hotelBookings[hotelBookings['country'].isna()][['hotel', 'is_canceled', 'country']] # Filter rows where 'country' is missing and show only the hotel, cancellation status, and country columns 

# hotelBookings['country'].fillna('Unknown', inplace=True)

hotelBookings['country'] = hotelBookings['country'].fillna('Unknown')
 

In [10]:
hotelBookings.drop(columns=['company'], inplace=True)

### Investigate Data Types 

- Use `.dtypes` attribute to investigate the types of data in each column.
- Change the types of data accordingly assiging the df to the result of the `astype()` function.
- `astype` will take a dictionary where each **key** is the column containing the wrong data type and each **value** is the new data type for that column.

In [ ]:
hotelBookings.dtypes

hotelBookings = hotelBookings.astype({'is_canceled': 'boolean',
                                      'is_repeated_guest': 'boolean',
                                      'count_of_children': 'int'
                                      })
hotelBookings.dtypes


### Bin Columns

Group columns using `pd.cut` function to group values. It requires the "bins" and the "labels" for each group. <br>
This function will lookup the value in a column and assign a value based on which bin it falls under

In [ ]:
# Check Unique values:
hotelBookings['lead_time'].unique()

# Use Describe to check for min and max values for a certain column:
hotelBookings['lead_time'].describe()

# Create bins by especifying the edges o the bin:
bins = [0, 100, 200, 300, 400, 500, 600, 700, 800]

# Create labels to specify ranges:
labels = ['0-100', '101-200', '201-300', '301-400', '401-500', '501-600', '601-700', '701-800']

# Use pd.cut to assign value to a new column:
hotelBookings['lead_time_binned'] = pd.cut(hotelBookings['lead_time'], bins=bins, labels=labels)

# Print outputs:
hotelBookings[['lead_time', 'lead_time_binned']]

### Separate Columns

Split a column into two columns using a delimiter using the `.split()` function. It returns an `array`.<br>
The `.split()` function will take two arguments in this case: the character to be used as a delimiter and `expand=True`.<br>
`expand=True` allocate each portion of the split string into a new column, which makes it easier to manipulate.<br>
`[0]` returns the first items of the array that results from this operation, in thsi case, the month.<br>
Use function `.pop()` to "pop" the column out of place and use the `.insert()` function to get into correct place.<br>
The `.insert(). function takes the `index` in which you want the column to be, the `column_name` to be moved, and `column_to_move` as arguments.



In [ ]:
# Create appropriate columns
# hotelBookings['arrival_date_month'] = hotelBookings['arrival_date'].str.split('-', expand=True)[0]
# hotelBookings['arrival_date_year'] = hotelBookings['arrival_date'].str.split('-', expand=True)[1]
hotelBookings.head()

# This is not making any sense.. The original table has these columns already separated.

### String Cleaning

Use `regex` to clean up strings.


In [ ]:
# Assign column to self and use the replace function to pass a regex as an argument:
hotelBookings['hotel'] = hotelBookings['hotel'].replace(r"[\*\n\^]", '', regex=True) #regex=True indicates this is a regex expression.

# Verify if values were replaced using .unique:
hotelBookings['hotel'].unique()


### Remove Duplicates

Use the `.duplicated()` function to remove values that are duplicates across all columns.<br>
Pass the `keep` parameter decides which occurrences of a duplicate should be marked as `True`.<br>
`keep=False` marks all rows that are duplicates as `True`, not just the later ones.<br>
Basically `keep=False` shows every row that is not unique.<br>
Use `.loc` to isolate and print only duplicates.<br>
Use `.drop_duplicates` to keep the first instance of a row and drop duplicates<br>
`.drop_duplicates()` takes a `subset` parameter that tells pandas which columns to look at when deciding what counts as a duplicate.<br>
If `subset` is not specified, pandas compares every column.<br>
Otherwise, pandas compares only those columns and ignores the rest.
Lastly, `.drop_duplicates` takes a `keep` parameter that takes `First` as value to specify that the first instance of a row should be kept


In [ ]:
# Return a boolean series in which True means a duplicate row:
hotelBookings.duplicated(keep=False)

# Use .loc tp print the rows that are duplicated:
hotelBookings.loc[hotelBookings.duplicated(keep=False)]

# Keep the first instance of the row and drop duplicates:
hotelBookings.drop_duplicates(keep='first', inplace=True)

# Checks if duplicates were truly removed:
hotelBookings.loc[hotelBookings.duplicated(keep=False)]